In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install librosa soundfile numpy sklearn pyaudio

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!pip install soundfile

In [ ]:
!pip install resampy

In [ ]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
import librosa.display
# Import resampy explicitly
import resampy

def extract_feature(file_name, mfcc, chroma, mel):
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    result = np.array([])

    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))

    if chroma:
        stft = np.abs(librosa.stft(X))
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
        result = np.hstack((result, chroma))

    if mel:
        mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)  # Corrected here
        result = np.hstack((result, mel))

    return result


# ... (rest of the code remains the same)

In [ ]:
emotions={
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

In [ ]:
  observed_emotions=['neutral','calm','happy','sad','angry','fearful','disgust','surprised']

In [ ]:
def load_data(test_size=0.2):
  x,y=[],[]
  for file in glob.glob("/content/drive/MyDrive/ML PBL/Actor_*/*.wav"):
    file_name=os.path.basename(file)
    emotion=emotions[file_name.split("-")[2]]
    if emotion not in observed_emotions:
      continue
    feature=extract_feature(file,mfcc=True,chroma=True,mel=True)
    x.append(feature)
    y.append(emotion)
  return train_test_split(np.array(x),y,test_size=test_size,train_size=0.75,random_state=9)

In [ ]:
import time
import resampy
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [23]:
print((x_train.shape[0],x_test.shape[0]))

(1080, 360)


In [24]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [25]:
model=MLPClassifier(alpha=0.01,batch_size=256,epsilon=1e-08,hidden_layer_sizes=(300,),learning_rate='adaptive',max_iter=500)

In [27]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [28]:
y_pred=model.predict(x_test)

In [29]:
accuracy=accuracy_score(y_true=y_test,y_pred=y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100.0))

Accuracy: 47.22%


In [30]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       angry       0.67      0.70      0.69        47
        calm       0.38      0.80      0.51        55
     disgust       0.48      0.28      0.36        46
     fearful       0.50      0.41      0.45        41
       happy       0.56      0.35      0.43        51
     neutral       0.20      0.04      0.06        28
         sad       0.44      0.40      0.42        52
   surprised       0.48      0.57      0.52        40

    accuracy                           0.47       360
   macro avg       0.46      0.45      0.43       360
weighted avg       0.48      0.47      0.45       360



In [31]:
from sklearn.metrics import confusion_matrix
matrix=confusion_matrix(y_test,y_pred)
print(matrix)

[[33  5  3  1  1  0  3  1]
 [ 0 44  3  3  0  0  5  0]
 [ 6 14 13  0  3  0  2  8]
 [ 2  5  1 17  4  2  7  3]
 [ 5  6  2 10 18  0  3  7]
 [ 0 20  3  0  0  1  4  0]
 [ 2 17  1  2  2  1 21  6]
 [ 1  6  1  1  4  1  3 23]]


In [47]:
# Load the necessary libraries
import numpy as np
import librosa

# Function to predict emotion from a new audio file
def predict_emotion(file_path, model):
    # Extract features from the new audio file
    features = extract_feature(file_path, mfcc=True, chroma=True, mel=True)

    # Reshape features to fit the model input
    features = features.reshape(1, -1)  # Reshape for single sample

    # Make prediction
    predicted_emotion = model.predict(features)
    # Add this inside your predict_emotion function

    return predicted_emotion[0]  # Return the predicted emotion

# Example usage
new_audio_file_path = '/content/03-01-03-02-01-01-05.wav'  # Replace with your audio file path

# Make sure to replace the above path with the correct path to your new audio file in Google Drive

predicted_emotion = predict_emotion(new_audio_file_path, model)
print(f"The predicted emotion for the new audio file is: {predicted_emotion}")


The predicted emotion for the new audio file is: surprised
